In [ ]:
import os
dir_names = []

for dirname, dirnames, filenames in os.walk('./archive.luftdaten.info'):
    # print path to all subdirectories first.
    for subdirname in dirnames:
        path = os.path.join(dirname, subdirname)
        dir_names.append(path)

In [ ]:
dir = "./archive.luftdaten.info/2017-08-27/"
df_result = spark.read.option("header", "true").option("delimiter", ";").csv(dir+"/*dht22*.csv")
for dir in dir_names:
    try:
        df = spark.read.option("header", "true").option("delimiter", ";").csv(dir+"/*dht22*.csv")
        
        df_result = df_result.union(df)
    except Exception as e:
        print "This path doesn't seem to contain dht22 sensor data: %s"%dir
        print e
    
    


In [ ]:
df_result.show()

In [ ]:
df_timestamp_temp = df_result.select('timestamp','temperature').withColumn("temp", df_result.temperature.cast("double")).drop("temperature")




In [ ]:
#!pip install pixiedust

In [ ]:
import pixiedust

Pixiedust database opened successfully


/usr/local/src/bluemix_jupyter_bundle.v61/notebook/lib/python2.7/site-packages/bokeh/core/json_encoder.py:52: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  NP_EPOCH = np.datetime64('1970-01-01T00:00:00Z')


In [ ]:
df_timestamp_temp.printSchema()

In [ ]:
df_timestamp_temp.count()

In [ ]:
from pyspark.sql import functions as F
format = "yyyy-MM-dd'T'HH:mm:ss"
df_ts_uts_temp = df_timestamp_temp.withColumn('ts', F.unix_timestamp('timestamp', format).cast('timestamp')).withColumn('uts', F.unix_timestamp('timestamp', format)).drop("timestamp")




In [ ]:
df_ts_uts_temp.show()

In [ ]:
display(df_ts_uts_temp)

In [ ]:
df_ts_uts_temp_add_feat = df_ts_uts_temp \
.withColumn("day_of_month",F.dayofmonth('ts')) \
.withColumn("dayof_year",F.dayofyear('ts')) \
.withColumn("month",F.month('ts'))


In [ ]:
df_ts_uts_temp_add_feat.show()

In [ ]:
display(df_ts_uts_temp_add_feat)

In [ ]:
df_ts_uts_temp_add_feat.write.parquet("archive.luftdaten.info.df_ts_uts_temp_add_feat.parquet")

In [1]:

import ibmos2spark

# @hidden_cell
credentials = {
    'auth_url': 'https://identity.open.softlayer.com',
    'project_id': '6aaf54352357483486ee2d4981f8ef15',
    'region': 'dallas',
    'user_id': '8659881f7846412188712d97ccee59b1',
    'username': 'member_da54c0bc59e0c60e5dde85e4c1298ebee56dd1b1',
    'password': 'B5A_IZ^8sG/D~-,i'
}

configuration_name = 'os_d3bd5b94a9334de59a55a7fed2bedeaa_configs'
bmos = ibmos2spark.bluemix(sc, credentials, configuration_name)

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/1.6.0/api/python/pyspark.sql.html#pyspark.sql.SQLContext
# The SQLContext object is already initalized for you.
# The following variable contains the path to your file on your Object Storage.
path_1 = bmos.url('ticont', 'part-00000-3884cac2-290d-431a-9a9e-04cbf1c384c4.snappy.parquet')



In [2]:
df_ts_uts_temp_add_feat_parquet = spark.read.parquet(path_1)

In [18]:
df_ts_uts_temp_add_feat_parquet = df_ts_uts_temp_add_feat_parquet.repartition(1)

In [19]:
#df_ts_uts_temp_add_feat_parquet.write.parquet("archive.luftdaten.info.df_ts_uts_temp_add_feat.1.parquet")

In [3]:
#hack/samples
df_ts_uts_temp_add_feat_parquet = df_ts_uts_temp_add_feat_parquet.sample(True, 0.01)

In [4]:
df_ts_uts_temp_add_feat_parquet_not_null = df_ts_uts_temp_add_feat_parquet \
.where(df_ts_uts_temp_add_feat_parquet.day_of_month.isNotNull()) \
.where(df_ts_uts_temp_add_feat_parquet.dayof_year.isNotNull()) \
.where(df_ts_uts_temp_add_feat_parquet.month.isNotNull()) \
.where(df_ts_uts_temp_add_feat_parquet.temp.isNotNull()) \
.withColumnRenamed("temp","label")

In [5]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler

categorical_columns= ['day_of_month', 'dayof_year', 'month']
encoders = [OneHotEncoder(dropLast=False,inputCol=column,
            outputCol="{0}_encoded".format(column)) 
    for column in categorical_columns
]

In [6]:
assembler = VectorAssembler(inputCols=[
  encoder.getOutputCol() for encoder in encoders
],outputCol="features")

In [7]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=encoders+[assembler])

In [8]:
model = pipeline.fit(df_ts_uts_temp_add_feat_parquet_not_null)

In [9]:
df_transformed = model.transform(df_ts_uts_temp_add_feat_parquet_not_null)

In [10]:
df_transformed.show()

+-----+--------------------+----------+------------+----------+-----+--------------------+------------------+--------------+--------------------+
|label|                  ts|       uts|day_of_month|dayof_year|month|day_of_month_encoded|dayof_year_encoded| month_encoded|            features|
+-----+--------------------+----------+------------+----------+-----+--------------------+------------------+--------------+--------------------+
| 17.6|2017-08-08 17:21:...|1502230862|           9|       221|    8|      (32,[9],[1.0])| (367,[221],[1.0])|(13,[8],[1.0])|(412,[9,253,407],...|
| 17.6|2017-08-08 17:32:...|1502231525|           9|       221|    8|      (32,[9],[1.0])| (367,[221],[1.0])|(13,[8],[1.0])|(412,[9,253,407],...|
| 17.5|2017-08-08 17:32:...|1502231543|           9|       221|    8|      (32,[9],[1.0])| (367,[221],[1.0])|(13,[8],[1.0])|(412,[9,253,407],...|
| 17.4|2017-08-08 17:48:...|1502232492|           9|       221|    8|      (32,[9],[1.0])| (367,[221],[1.0])|(13,[8],[1.0])|

In [11]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [12]:
gbt = GBTRegressor(featuresCol="features", maxIter=10)



In [ ]:
# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[gbt])



In [ ]:
# Train model.  This also runs the indexer.
model = pipeline.fit(df_transformed)



In [ ]:
# Make predictions.
predictions = model.transform(df_transformed)



In [ ]:
df_transformed.printSchema()

root
 |-- label: double (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- uts: long (nullable = true)
 |-- day_of_month: integer (nullable = true)
 |-- dayof_year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day_of_month_encoded: vector (nullable = true)
 |-- dayof_year_encoded: vector (nullable = true)
 |-- month_encoded: vector (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
display(predictions)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|21.993505198649427| 17.5|(412,[9,253,407],...|
|21.993505198649427| 17.6|(412,[9,253,407],...|
|21.993505198649427| 17.6|(412,[9,253,407],...|
|21.993505198649427| 17.5|(412,[9,253,407],...|
|21.993505198649427| 17.5|(412,[9,253,407],...|
+------------------+-----+--------------------+
only showing top 5 rows



In [35]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)



Root Mean Squared Error (RMSE) on test data = 244.088


In [36]:
gbtModel = model.stages[0]
print(gbtModel)  # summary only

GBTRegressionModel (uid=GBTRegressor_4b9e99769b26b2794ad5) with 10 trees


In [ ]:
predictions = model.transform(df_transformed)

In [45]:
single = spark.createDataFrame(df_transformed.rdd.take(1))

In [46]:
single.show()

+-----+--------------------+----------+------------+----------+-----+--------------------+------------------+--------------+--------------------+
|label|                  ts|       uts|day_of_month|dayof_year|month|day_of_month_encoded|dayof_year_encoded| month_encoded|            features|
+-----+--------------------+----------+------------+----------+-----+--------------------+------------------+--------------+--------------------+
| 17.5|2017-08-09 00:27:...|1502231227|           9|       221|    8|      (32,[9],[1.0])| (367,[221],[1.0])|(13,[8],[1.0])|(412,[9,253,407],...|
+-----+--------------------+----------+------------+----------+-----+--------------------+------------------+--------------+--------------------+



In [47]:
predictions = model.transform(single)

In [48]:
predictions.show()

+-----+--------------------+----------+------------+----------+-----+--------------------+------------------+--------------+--------------------+------------------+
|label|                  ts|       uts|day_of_month|dayof_year|month|day_of_month_encoded|dayof_year_encoded| month_encoded|            features|        prediction|
+-----+--------------------+----------+------------+----------+-----+--------------------+------------------+--------------+--------------------+------------------+
| 17.5|2017-08-09 00:27:...|1502231227|           9|       221|    8|      (32,[9],[1.0])| (367,[221],[1.0])|(13,[8],[1.0])|(412,[9,253,407],...|21.993505198649427|
+-----+--------------------+----------+------------+----------+-----+--------------------+------------------+--------------+--------------------+------------------+

